In [1]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel

torch.random.manual_seed(0)

model_path = "Qwen/Qwen3-4B-Instruct-2507"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto",
)
model = PeftModel.from_pretrained(model, 'qwen3_lora_finetuned/checkpoint-4235')

tokenizer = AutoTokenizer.from_pretrained(model_path)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [22]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": True,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 20,
    "min_p": 0,
}

def get_prompt():
    return [
        {
            "role": "user", 
            "content": "Write a satirical headline in the style of Onion News. Only write the headline."
        }
    ]

output = pipe(get_prompt(), num_return_sequences=5, **generation_args)
output

/home/joshuachin/miniconda3/envs/code-review/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


[{'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News. Only write the headline.'},
   {'role': 'assistant',
    'content': '<think>\n\n</think>\n\nReport: 34% Of Americans Have No Idea How To Use Gun'}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News. Only write the headline.'},
   {'role': 'assistant',
    'content': "<think>\n\n</think>\n\nNew York City Mayor: 'We Will Continue To Have The Largest Population Of Gay People In The World'"}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News. Only write the headline.'},
   {'role': 'assistant',
    'content': '</tool_call>\n\n</tool_call>\n\nIowa Governor: ‘I Will Not Be Stopped By A Few Fucks’'}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News. Only write the headline.'},
   {'role': 'assistant',
    'c

In [14]:
prompt = get_prompt()
prompt.append({
    "role": "assistant",
    "content": "nTrump Campaign Announces It Will Stop Allowing Women On Campaign Bus",
})

tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
)

'<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nnTrump Campaign Announces It Will Stop Allowing Women On Campaign Bus<|im_end|>\n'

In [15]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 

In [17]:
from datasets import load_dataset

ds = load_dataset("Biddls/Onion_News")
ds['train'][0]

{'text': 'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage #~# LOS ANGELES—Admitting that she’d made some major lifestyle changes since developing her famous KonMari method, a relaxed Marie Kondo told reporters Tuesday that she was now perfectly happy living in waist-high sewage. “The truth is, while I used to be very hard on myself about keeping everything clean, I’m now able to find peace living my life half-submerged in a large, fetid pool of human waste,” said Kondo, who added that while things like tidiness, organization, and minimalism used to spark joy for her, she now felt that same warmth from wading, floating, and swimming in the many gallons of untreated urine and feces that currently filled her home. “It was difficult, but once I had my children, I began to find it impossible to remove, clean, and sanitize the unending stream of excrement that bubbled up out of my toilets and filled my house to the point of collapse. While I used to hate it, I no

In [18]:
SEP = '#~#'

def get_as_messages(data):
    headline = get_headline(data['text'])
    messages = get_prompt()
    messages.append(format_reply(headline))
    return {'messages': messages}
    
def get_headline(text):
    return text.split(SEP)[0].strip()

def get_prompt():
    return [
        {
            "role": "user", 
            "content": "Write a satirical headline in the style of Onion News. Only write the headline."
        }
    ]

def format_reply(headline):
    return {
        "role": "assistant",
        "content": headline,
    }

messages_ds = ds.map(get_as_messages, remove_columns=['text'])
messages_ds['train'][0]

{'messages': [{'content': 'Write a satirical headline in the style of Onion News. Only write the headline.',
   'role': 'user'},
  {'content': 'Relaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage',
   'role': 'assistant'}]}

In [21]:
tokenizer.apply_chat_template(
    messages_ds['train'][:10]['messages'],
    tokenize=False,
)

['<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n',
 '<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nU.S. Officials Call For Correct Amount Of Violence<|im_end|>\n',
 '<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nKamala Harris Asks Communications Assistant If She Can Take Them Out For Coffee And Pick Their Brain Sometime<|im_end|>\n',
 '<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n25 Arrested In Fake Nursing School Diploma Scheme<|im_end|>\n',
 '<|im_start|>user\nWrite a 